In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, LSTM, Activation, Masking, InputLayer
from sklearn.preprocessing import MinMaxScaler

In [134]:
n = 1000
length = 180
pruning_ratio = 0.95
seqs = []
pruning_value = -100
for _ in range(n):
  start, step = np.random.randint(1, 100, 2)
  seq = np.arange(start, start+step*length+1 , step)
  seqs.append(seq)
seqs = np.asarray(seqs)
# seqs = (seqs - seqs.min()) / (seqs.max() - seqs.min())
seqs = (seqs - seqs.mean()) / seqs.std()
xs = np.asarray(seqs[:, :-1])
query = np.random.randint(0, length, (n, int(pruning_ratio*length)))
for seq, q in zip(xs, query):
  seq[q] = pruning_value
xs = np.expand_dims(xs, axis=2)
ys = np.asarray(seqs[:, -1])
# ys = (ys - ys.mean()) / ys.std()
# ys = (ys - ys.min()) / (ys.max() - ys.min())
ys = np.expand_dims(ys, axis=2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [135]:
xs.shape, ys.shape

((1000, 180, 1), (1000, 1))

In [136]:
model = Sequential()
RNN = CuDNNLSTM
base_activation = 'tanh'
model.add(InputLayer((length, 1)))
# model.add(Masking(-1))
model.add(RNN(32, return_sequences=True,))
model.add(Activation(base_activation))
model.add(RNN(32,))
model.add(Activation(base_activation))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_22 (CuDNNLSTM)    (None, 180, 32)           4480      
_________________________________________________________________
activation_28 (Activation)   (None, 180, 32)           0         
_________________________________________________________________
cu_dnnlstm_23 (CuDNNLSTM)    (None, 32)                8448      
_________________________________________________________________
activation_29 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 12,961
Trainable params: 12,961
Non-trainable params: 0
_________________________________________________________________


In [139]:
model.fit(xs, ys, epochs=100, shuffle=True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 2.9058e-04 - val_loss: 1.8469e-04
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 3.0647e-04 - val_loss: 5.1687e-04
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 3.0810e-04 - val_loss: 0.0010
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 3.0502e-04 - val_loss: 1.8792e-04
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 1.4957e-04 - val_loss: 2.1053e-04
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 2.2912e-04 - val_loss: 1.4488e-04
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 1.7032e-04 - val_loss: 2.7794e-04
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 1.8399e-04 - val_loss: 2.4895e-04
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 1.7209e-04 